In [ ]:
import pandas as pd
from pathlib import Path
import plotly.express as px
from plotly.io import to_html
from plotly.subplots import make_subplots
import plotly.graph_objects as go


df=pd.read_csv('nba_dataset/games_details_regular.csv')

team_stats=(df
            .groupby(['TEAM','GAME_ID'])[['FGM','FGA','FTA','OREB','TO','MIN']]
            .sum()
            .reset_index())

team_stats['team_poss']=(
    team_stats['FGA']+
    (0.44*team_stats['FTA'])-
    team_stats['OREB']+
    team_stats['TO']
).round(2)

team_stats=team_stats.rename(columns={'MIN':'team_minutes'})
team_stats=team_stats.rename(columns={'FGM':'team_FGM'})
team_stats['team_minutes']=team_stats['team_minutes'].round(2)

df['MIN']=df['MIN'].round(2)
details = df.merge(
    team_stats[['GAME_ID','TEAM','team_poss','team_minutes','team_FGM']],
    on=['GAME_ID','TEAM'],
    how='left'
)

# 3. USG% 계산
details['USG%'] = 100 * (
    (details['FGA'] + (0.44*details['FTA']) + details['TO']) * details['team_minutes']
) / (details['MIN'] * details['team_poss'])

details['USG%'] = details['USG%'].round(2)

details['player_poss'] = (
    details['FGA'] + (0.44*details['FTA']) + details['TO']
)

details['PTS_per_poss'] = (details['PTS'] / details['player_poss']).round(2)

details['TS%']=details['PTS']/(2*(details['FGA']+0.44*details['FTA']))
details['TS%']=details['TS%'].round(2)

details['TO%']=details['TO']/(details['FGA']+0.44*details['FTA']+details['TO'])
details['TO%']=details['TO%'].round(2)
df1 = details.copy()


# BBR 공식: AST% = 100 * AST / [ ((MIN / (team_minutes/5)) * team_FGM) - FGM ]
den = ((df1['MIN'] / (df1['team_minutes'] / 5.0)) * df1['team_FGM']) - df1['FGM']
df1['AST_pct_bbr'] = (100.0 * df1['AST'] / den.replace(0, np.nan)).clip(0, 100)
df1['AST_pct_bbr']=df1['AST_pct_bbr'].round(2)

players=pd.read_csv('nba_dataset/games_details.csv')

player = players[['PLAYER_NAME','PLAYER_ID']]

# 필요한 컬럼만 남긴 매핑 테이블
player = players[['PLAYER_NAME','PLAYER_ID']]

# df1과 합치기 (PLAYER_ID 기준)
df1_named = df1.merge(player, on='PLAYER_ID', how='left')

# -----------------------------
# 1) USG% 계산 (possession 기반, 올바른 스케일)
#    USG% = 100 * (FGA + 0.44*FTA + TO) * 48 / (MIN * team_poss)
#    ※ team_poss 를 분모로 쓸 때는 48(= 팀 평균 개인출전시간)로 스케일링해야 함
# -----------------------------
num_cols = ['FGA','FTA','TO','MIN','team_poss']
for c in num_cols:
    df1_named[c] = pd.to_numeric(df1_named[c], errors='coerce')

num = (df1_named['FGA'] + 0.44*df1_named['FTA'] + df1_named['TO']) * 48
den = (df1_named['MIN'] * df1_named['team_poss'])

df1_named['USG%'] = np.where(den > 0, 100 * num / den, np.nan).round(2)

# -----------------------------
# 2) 디트로이트 "선발(C/F/G)"만 필터
#    (팀 컬럼명이 다른 경우: TEAM_ABBREVIATION=='DET' 등으로 바꿔주세요)
# -----------------------------
valid_pos = ['C','F','G']
det_details = df1_named[
    (df1_named['TEAM'] == 'Detroit') &
    (df1_named['START_POSITION'].isin(valid_pos))
].copy()

# -----------------------------
# 3) 시간가중 USG (USG% × MIN) 집계
# -----------------------------
det_details['USG_x_MIN'] = det_details['USG%'] * det_details['MIN']

player_pos_usg = (
    det_details
      .groupby(['PLAYER_ID','PLAYER_NAME','START_POSITION'], observed=True)
      .agg(
          USG_minutes_sum = ('USG_x_MIN','sum'),    # Σ(USG% × MIN)
          MIN_sum         = ('MIN','sum'),          # Σ MIN
          games           = ('GAME_ID','nunique')   # 경기 수
      )
      .assign(
          USG_wavg = lambda d: np.where(d['MIN_sum']>0,
                                        d['USG_minutes_sum']/d['MIN_sum'],
                                        np.nan)
      )
      .reset_index()
      .sort_values(['START_POSITION','USG_wavg'], ascending=[True, False])
)

# 보기 좋게 반올림
player_pos_usg['USG_wavg']       = player_pos_usg['USG_wavg'].round(2)
player_pos_usg['USG_minutes_sum'] = player_pos_usg['USG_minutes_sum'].round(2)

#------------------------------------------------------------------------------------

# 색상 팔레트 (큰 비중부터 차례대로)
custom_colors = ["#d73027", "#4575b4", "#f7f7f7", "#000000"]

positions = ['C','F','G']
subfig = make_subplots(
    rows=1, cols=len(positions),
    specs=[[{"type":"domain"}]*len(positions)],
    subplot_titles=positions
)

for i, pos in enumerate(positions, start=1):
    subdf = player_pos_usg[player_pos_usg['START_POSITION'] == pos]

    # 비중(USG_minutes_sum) 큰 순서대로 정렬
    subdf = subdf.sort_values("USG_minutes_sum", ascending=False).reset_index(drop=True)

    # 색상: 큰 비중부터 빨강-파랑-흰-검정 순서로 할당
    colors = custom_colors * (len(subdf) // len(custom_colors) + 1)

    subfig.add_trace(
        go.Pie(
            labels=subdf['PLAYER_NAME'],
            values=subdf['USG_minutes_sum'],
            name=pos,
            textinfo="percent",
            textposition="inside",
            showlegend=True,
            marker=dict(colors=colors[:len(subdf)])
        ),
        row=1, col=i
    )
subfig.update_layout(
    title_text="DET 포지션별: 선수별 공격권 비중 (USG%×MIN)",
    legend_title="선수"
)




# 디트로이트 팀만
det = df1_named[df1_named['TEAM'] == 'Detroit'].copy()

# 선수별 평균 PPP 구하기
det_player = det.groupby('PLAYER_NAME')['PTS_per_poss'].mean().round(2).reset_index()
print(det_player.info())
det_player['PLAYER_NAME'] = det_player['PLAYER_NAME'].astype(str)
det1=det_player.sort_values('PTS_per_poss', ascending=False)
# 막대그래프 그리기
fig = px.bar(det1,
             x='PLAYER_NAME',
             y='PTS_per_poss',
             title="Detroit 선수별 평균 PTS per Poss",
            )


# 1) 상위팀만 필터
top = df1_named[df1_named['TEAM_LVL'] == '상위팀'].copy()

# 3) 팀별 총합 (득점/포제션)
top_team = top.groupby('TEAM', as_index=False).agg(
    PTS_sum=('PTS','sum'),
    poss_sum=('player_poss','sum')
)
top_team['PPP'] = np.where(top_team['poss_sum']>0,
                           top_team['PTS_sum']/top_team['poss_sum'], 0)

print("=== 상위팀별 PPP ===")
print(top_team[['TEAM','PPP']].sort_values('PPP', ascending=False))

# 4) 상위팀 평균 PPP
top_avg_ppp = top_team['PPP'].mean()
print("\n=== 상위팀 평균 PPP ===")
print(f"{top_avg_ppp:.3f}")

# 1) 디트로이트만 필터
pos = ['C','F','G']
det_starters = df1[
    (df1['TEAM'].str.contains("Detroit", case=False, na=False)) &
    (df1['START_POSITION'].isin(pos))
].copy()


# 2) 숫자형 변환
for c in ['PTS','player_poss']:
    det[c] = pd.to_numeric(det[c], errors='coerce').fillna(0)

# 3) 팀 전체 PPP = 총득점 / 총포제션
det_total_pts = det['PTS'].sum()
det_total_poss = det['player_poss'].sum()

det_ppp = det_total_pts / det_total_poss if det_total_poss > 0 else 0

print("=== 디트로이트 팀 PPP ===")
print(f"총득점: {det_total_pts}, 총포제션: {det_total_poss}")
print(f"팀 PPP: {det_ppp:.3f}")

print(f"디트로이트 PPP: {det_ppp:.3f}")
print(f"상위팀 평균 PPP: {top_avg_ppp:.3f}")


# 경기+팀별 포제션 합
game_team_poss = df1.groupby(['GAME_ID','TEAM'], as_index=False)['player_poss'].sum()

# 팀별 경기 포제션 평균
avg_poss = game_team_poss['player_poss'].mean()
print("팀당 경기당 평균 포제션:", f"{avg_poss:.3f}")

data={
    '디트로이트 ppp':0.978,
    '상위팀 평균 ppp':1.028
}
ppp=pd.Series(data).reset_index()
ppp.columns = ['팀', 'PPP']
fig2=px.bar(
    ppp,
    x='팀',
    y='PPP' ,
    title="상위팀과 디트로이트 팀의 PTS per Possession 비교")


# 1) 상위팀 + 선발(C/F/G)만 필터
pos = ['C','F','G']
top_starters = df1_named[(df1_named['TEAM_LVL'] == '상위팀') &
                   (df1_named['START_POSITION'].isin(pos))].copy()

# 2) 상위팀 선발 전체의 TS% 평균
top_starters_ts = top_starters['TS%'].mean()

print("상위팀 선발 평균 TS%:", round(top_starters_ts, 3))

# 1) 선발 포지션 정의
pos = ['C','F','G']

# 2) 디트로이트 팀 + 선발만 필터
det_starters = df1_named[
    (df1_named['TEAM']=="Detroit") &
    (df1_named['START_POSITION'].isin(pos))
].copy()

# 3) TS% 평균 계산
det_starters_ts = det_starters['TS%'].mean()

print("디트로이트 선발 평균 TS%:", round(det_starters_ts, 3))

data={
    '디트로이트 선발 평균 TS%': 0.561,
    '상위팀 선발 평균 TS%': 0.591
}
ts=pd.Series(data).reset_index()
ts.columns = ['팀', 'TS%']
fig3=px.bar(
    ts,
    x='팀',
    y='TS%',
    title="상위팀 vs 디트로이트 팀 TS% 비교"
)

pos = ['C','F','G']
det_starters = df1_named[
    (df1_named['TEAM']=="Detroit") &
    (df1_named['START_POSITION'].isin(pos))
].copy()

# 선수별 TS% 평균 (포지션 포함)
det_player_ts = det_starters.groupby(['PLAYER_NAME','START_POSITION'], as_index=False)['TS%'].mean()

# 산점도 (TS% 크기로 반영)
fig4 = px.scatter(
    det_player_ts,
    x='PLAYER_NAME',
    y='TS%',
    color='START_POSITION',   # 포지션별 색 구분
    size='TS%',
    title="디트로이트 선발 선수별 TS% 분포 (포지션별 구분)",
    labels={'START_POSITION': 'Position'},   # 범례 제목 바꾸기
    category_orders={'START_POSITION': ['C','F','G']}  # 범례 순서 고정
)

# 범례 항목별 레이블 직접 바꾸기
fig4.for_each_trace(lambda t: t.update(
    name = t.name.replace("C","Center").replace("F","Forward").replace("G","Guard")
))

# 선발 포지션 정의
pos = ['C','F','G']

# 상위팀 + 선발만 필터
top_starters = df1_named[
    (df1_named['TEAM_LVL'] == '상위팀') &
    (df1_named['START_POSITION'].isin(pos))
].copy()

# 선수별 평균 TS% 계산
top_player_ts = top_starters.groupby(['PLAYER_NAME','START_POSITION'], as_index=False)['TS%'].mean()


# 산점도
fig5 = px.scatter(
    top_player_ts,
    x='PLAYER_NAME',
    y='TS%',
    color='START_POSITION',   # 포지션별 색 구분
    size='TS%',               # TS% 크기 반영 (시각적 강조)
    hover_data=['START_POSITION'],
    title="상위팀 선발 선수별 TS% 분포"
)

fig5.for_each_trace(lambda t: t.update(name = t.name.replace("C","Center").replace("F","Forward").replace("G","Guard")))
fig5.update_traces(marker=dict(line=dict(width=1, color='black')))
fig5.update_layout(xaxis_tickangle=45, yaxis_title="TS%", width=1000, height=600)

det = df1_named[df1_named['TEAM'].str.contains("Detroit", case=False, na=False)].copy()

# 상위팀만 필터
top = df1_named[df1_named['TEAM_LVL'] == '상위팀'].copy()

# 평균 TO% 계산
det_to_mean = det['TO%'].mean()
top_to_mean = top['TO%'].mean()

print("디트로이트 평균 TO%:", round(det_to_mean, 3))
print("상위팀 평균 TO%:", round(top_to_mean, 3))

data = {
    '팀': ['디트로이트', '상위팀'],
    'TO% 평균': [det_to_mean, top_to_mean]
}
df_plot = pd.DataFrame(data)

fig6 = px.bar(df_plot,
             x='팀',
             y='TO% 평균',
             title="디트로이트 vs 상위팀 TO% 평균 비교",
             text='TO% 평균')

fig6.update_traces(texttemplate='%{text:.3f}', textposition='outside')
fig6.update_yaxes(range=[0,0.15])

# 선발 포지션
pos = ['C','F','G']

# 디트로이트 선발 선수만
det_starters = df1_named[
    (df1_named['TEAM']=="Detroit") &
    (df1_named['START_POSITION'].isin(pos))
]

# 선수별 평균 TO%
det_starters_to = det_starters.groupby('PLAYER_NAME', as_index=False)['TO%'].mean()

# 시각화
fig7 = px.bar(det_starters_to,
             x='PLAYER_NAME',
             y='TO%',
             title="디트로이트 선발 선수별 평균 TO%",
             text='TO%')
fig7.update_traces(texttemplate='%{text:.3f}', textposition='outside')
fig7.update_layout(xaxis_tickangle=45)
fig7.update_yaxes(range=[0,0.25])
import plotly.express as px

# 디트로이트 선수만 추출
pos = ['C','F','G']

# 디트로이트 선발 선수만
det= df1_named[
    (df1_named['TEAM']=="Detroit") &
    (df1_named['START_POSITION'].isin(pos))
]
# 선수별 평균 AST%
det_ast = det.groupby(['PLAYER_NAME','START_POSITION'], as_index=False)['AST%'].mean()

# 산점도
fig = px.scatter(det_ast,
                 x='PLAYER_NAME',
                 y='AST%',
                 color='START_POSITION',  # 포지션별 색 구분
                 title="디트로이트 선수별 AST% 분포 (포지션별)",
                 size='AST%',             # 점 크기로 기여도 표현
                 hover_data=['START_POSITION'])

fig.update_layout(
    xaxis_tickangle=45,
    yaxis=dict(title="AST%"),
    width=950,
    height=600
)



# 디트로이트 팀만 필터
det = df1_named[df1_named['TEAM']=="Detroit"].copy()

# 상위팀만 필터
top = df1_named[df1_named['TEAM_LVL'] == '상위팀'].copy()

# 평균 AST% 계산
det_ast_mean = det['AST%'].mean()
top_ast_mean = top['AST%'].mean()

print("디트로이트 평균 AST%:", round(det_ast_mean, 3))
print("상위팀 평균 AST%:", round(top_ast_mean, 3))

# 시각화
data = {
    '팀': ['디트로이트', '상위팀'],
    'AST% 평균': [det_ast_mean, top_ast_mean]
}
df_plot = pd.DataFrame(data)

fig8 = px.bar(df_plot,
             x='팀',
             y='AST% 평균',
             title="디트로이트 vs 상위팀 AST% 평균 비교",
             text='AST% 평균')

fig8.update_traces(texttemplate='%{text:.3f}', textposition='outside')
fig8.update_yaxes(range=[0,7])

# 디트로이트 선수만 추출
pos = ['C','F','G']

# 디트로이트 선발 선수만
det= df1_named[
    (df1_named['TEAM']=="Detroit") &
    (df1_named['START_POSITION'].isin(pos))
]
# 선수별 평균 AST%
det_ast = det.groupby(['PLAYER_NAME','START_POSITION'], as_index=False)['AST%'].mean()

# 산점도
fig9 = px.scatter(det_ast,
                 x='PLAYER_NAME',
                 y='AST%',
                 color='START_POSITION',  # 포지션별 색 구분
                 title="디트로이트 선수별 AST% 분포 (포지션별)",
                 size='AST%',             # 점 크기로 기여도 표현
                 hover_data=['START_POSITION'])

fig9.update_layout(
    xaxis_tickangle=45,
    yaxis=dict(title="AST%"),
    width=950,
    height=600
)

import plotly.express as px

# 선발 포지션 정의
pos = ['C','F','G']

# 상위팀 + 선발만 필터
top_starters = df1_named[
    (df1_named['TEAM_LVL'] == '상위팀') &
    (df1_named['START_POSITION'].isin(pos))
].copy()

# 선수별 평균 AST% 계산
top_player_ast = top_starters.groupby(['PLAYER_NAME','START_POSITION'], as_index=False)['AST%'].mean()

# 산점도
fig10 = px.scatter(
    top_player_ast,
    x='PLAYER_NAME',
    y='AST%',
    color='START_POSITION',   # 포지션별 색 구분
    size='AST%',              # AST% 크기 반영
    hover_data=['START_POSITION'],
    title="상위팀 선발 선수별 AST% 분포"
)

fig10.update_traces(marker=dict(line=dict(width=1, color='black')))
fig10.update_layout(xaxis_tickangle=45, yaxis_title="AST%", width=1000, height=600)


''' 여기까지가 시각화 작업'''


"=================== html로 내보내기 ======================="
# 첫 그래프 (뼈대)
html: str = to_html(fig1, include_plotlyjs='cdn', full_html=True)

parts = [
    to_html(fig2, include_plotlyjs=False, full_html=False),
    to_html(fig3, include_plotlyjs=False, full_html=False),
]

# 중간에 삽입
insertion = "\n<hr>\n".join(parts)
lower = html.lower()
insert_at = lower.rfind("</body>")
html = html[:insert_at] + insertion + html[insert_at:]

out_path ='index.html'
Path(out_path).write_text(html, encoding='utf-8')

print(f"Saved: {out_path}")

/var/folders/vf/9p8syxcx1g77s8jzp8b8b8fh0000gn/T/ipykernel_78954/2163936749.py:60: DtypeWarning:

Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17 entries, 0 to 16
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PLAYER_NAME   17 non-null     object 
 1   PTS_per_poss  16 non-null     float64
dtypes: float64(1), object(1)
memory usage: 400.0+ bytes
None
=== 상위팀별 PPP ===
           TEAM       PPP
1      Brooklyn  1.099763
3        Denver  1.062903
0        Boston  1.062050
2     Cleveland  1.033071
7  Philadelphia  1.026947
6   New Orleans  1.001894
5     Milwaukee  0.997720
4       Memphis  0.976648

=== 상위팀 평균 PPP ===
1.033
=== 디트로이트 팀 PPP ===
총득점: 1259524, 총포제션: 1215905.52
팀 PPP: 1.036
디트로이트 PPP: 1.036
상위팀 평균 PPP: 1.033
팀당 경기당 평균 포제션: 112.503
상위팀 선발 평균 TS%: 0.6
디트로이트 선발 평균 TS%: 0.579
디트로이트 평균 TO%: 0.121
상위팀 평균 TO%: 0.14


KeyError: 'AST%'

In [8]:
import pandas as pd
import numpy as np  # ✅ 추가
from pathlib import Path
import plotly.express as px
from plotly.io import to_html
from plotly.subplots import make_subplots
import plotly.graph_objects as go


df=pd.read_csv('nba_dataset/games_details_regular.csv')

team_stats=(df
            .groupby(['TEAM','GAME_ID'])[['FGM','FGA','FTA','OREB','TO','MIN']]
            .sum()
            .reset_index())

team_stats['team_poss']=(
    team_stats['FGA']+
    (0.44*team_stats['FTA'])-
    team_stats['OREB']+
    team_stats['TO']
).round(2)

team_stats=team_stats.rename(columns={'MIN':'team_minutes'})
team_stats=team_stats.rename(columns={'FGM':'team_FGM'})
team_stats['team_minutes']=team_stats['team_minutes'].round(2)

df['MIN']=df['MIN'].round(2)
details = df.merge(
    team_stats[['GAME_ID','TEAM','team_poss','team_minutes','team_FGM']],
    on=['GAME_ID','TEAM'],
    how='left'
)

# 3. USG% 계산 (네가 쓰던 식 그대로 둠)
details['USG%'] = 100 * (
    (details['FGA'] + (0.44*details['FTA']) + details['TO']) * details['team_minutes']
) / (details['MIN'] * details['team_poss'])

details['USG%'] = details['USG%'].round(2)

details['player_poss'] = (
    details['FGA'] + (0.44*details['FTA']) + details['TO']
)

details['PTS_per_poss'] = (details['PTS'] / details['player_poss']).round(2)

details['TS%']=details['PTS']/(2*(details['FGA']+0.44*details['FTA']))
details['TS%']=details['TS%'].round(2)

details['TO%']=details['TO']/(details['FGA']+0.44*details['FTA']+details['TO'])
details['TO%']=details['TO%'].round(2)
df1 = details.copy()

# BBR 공식: AST% = 100 * AST / [ ((MIN / (team_minutes/5)) * team_FGM) - FGM ]
den = ((df1['MIN'] / (df1['team_minutes'] / 5.0)) * df1['team_FGM']) - df1['FGM']
df1['AST_pct_bbr'] = (100.0 * df1['AST'] / den.replace(0, np.nan)).clip(0, 100)
df1['AST_pct_bbr']=df1['AST_pct_bbr'].round(2)
df1['AST%'] = df1['AST_pct_bbr']
players=pd.read_csv('nba_dataset/games_details.csv')

player = players[['PLAYER_NAME','PLAYER_ID']]

# df1과 합치기 (PLAYER_ID 기준)
df1_named = df1.merge(player, on='PLAYER_ID', how='left')

# -----------------------------
# 1) USG% (possession 기반 스케일) — 네가 원하는 대로 유지하려면 아래 블록은 그대로 둬도 됨
# -----------------------------
num_cols = ['FGA','FTA','TO','MIN','team_poss']
for c in num_cols:
    df1_named[c] = pd.to_numeric(df1_named[c], errors='coerce')

num = (df1_named['FGA'] + 0.44*df1_named['FTA'] + df1_named['TO']) * 48
den = (df1_named['MIN'] * df1_named['team_poss'])
df1_named['USG%'] = np.where(den > 0, 100 * num / den, np.nan).round(2)

# -----------------------------
# 2) 디트로이트 "선발(C/F/G)"만 필터
# -----------------------------
valid_pos = ['C','F','G']
det_details = df1_named[
    (df1_named['TEAM'] == 'Detroit') &
    (df1_named['START_POSITION'].isin(valid_pos))
].copy()

# -----------------------------
# 3) 시간가중 USG (USG% × MIN) 집계
# -----------------------------
det_details['USG_x_MIN'] = det_details['USG%'] * det_details['MIN']

player_pos_usg = (
    det_details
      .groupby(['PLAYER_ID','PLAYER_NAME','START_POSITION'], observed=True)
      .agg(
          USG_minutes_sum = ('USG_x_MIN','sum'),    # Σ(USG% × MIN)
          MIN_sum         = ('MIN','sum'),          # Σ MIN
          games           = ('GAME_ID','nunique')   # 경기 수
      )
      .assign(
          USG_wavg = lambda d: np.where(d['MIN_sum']>0,
                                        d['USG_minutes_sum']/d['MIN_sum'],
                                        np.nan)
      )
      .reset_index()
      .sort_values(['START_POSITION','USG_wavg'], ascending=[True, False])
)

# 보기 좋게 반올림
player_pos_usg['USG_wavg']       = player_pos_usg['USG_wavg'].round(2)
player_pos_usg['USG_minutes_sum'] = player_pos_usg['USG_minutes_sum'].round(2)

#------------------------------------------------------------------------------------

# 색상 팔레트 (큰 비중부터 차례대로)
custom_colors = ["#d73027", "#4575b4", "#f7f7f7", "#000000"]

positions = ['C','F','G']
subfig = make_subplots(
    rows=1, cols=len(positions),
    specs=[[{"type":"domain"}]*len(positions)],
    subplot_titles=positions
)

for i, pos in enumerate(positions, start=1):
    subdf = player_pos_usg[player_pos_usg['START_POSITION'] == pos]

    # 비중(USG_minutes_sum) 큰 순서대로 정렬
    subdf = subdf.sort_values("USG_minutes_sum", ascending=False).reset_index(drop=True)

    # 색상: 큰 비중부터 빨강-파랑-흰-검정 순서로 할당
    colors = custom_colors * (len(subdf) // len(custom_colors) + 1)

    subfig.add_trace(
        go.Pie(
            labels=subdf['PLAYER_NAME'],
            values=subdf['USG_minutes_sum'],
            name=pos,
            textinfo="percent",
            textposition="inside",
            showlegend=True,
            marker=dict(colors=colors[:len(subdf)])
        ),
        row=1, col=i
    )
subfig.update_layout(
    title_text="DET 포지션별: 선수별 공격권 비중 (USG%×MIN)",
    legend_title="선수"
)

# ✅ HTML의 뼈대 그래프로 쓸 변수 지정
fig1 = subfig  # 🔧 정의되지 않은 fig1 에러 해결

# 디트로이트 팀만
det = df1_named[df1_named['TEAM'] == 'Detroit'].copy()

# 선수별 평균 PPP 구하기
det_player = det.groupby('PLAYER_NAME')['PTS_per_poss'].mean().round(2).reset_index()
print(det_player.info())
det_player['PLAYER_NAME'] = det_player['PLAYER_NAME'].astype(str)
det1=det_player.sort_values('PTS_per_poss', ascending=False)
# 막대그래프 그리기
fig = px.bar(det1,
             x='PLAYER_NAME',
             y='PTS_per_poss',
             title="Detroit 선수별 평균 PTS per Poss",
            )

# 1) 상위팀만 필터 (TEAM_LVL 있을 때만)
if 'TEAM_LVL' in df1_named.columns:
    top = df1_named[df1_named['TEAM_LVL'] == '상위팀'].copy()

    # 3) 팀별 총합 (득점/포제션)
    top_team = top.groupby('TEAM', as_index=False).agg(
        PTS_sum=('PTS','sum'),
        poss_sum=('player_poss','sum')
    )
    top_team['PPP'] = np.where(top_team['poss_sum']>0,
                               top_team['PTS_sum']/top_team['poss_sum'], 0)

    print("=== 상위팀별 PPP ===")
    print(top_team[['TEAM','PPP']].sort_values('PPP', ascending=False))

    # 4) 상위팀 평균 PPP
    top_avg_ppp = top_team['PPP'].mean()
    print("\n=== 상위팀 평균 PPP ===")
    print(f"{top_avg_ppp:.3f}")

# 1) 디트로이트만 필터
pos = ['C','F','G']
det_starters = df1[
    (df1['TEAM'].str.contains("Detroit", case=False, na=False)) &
    (df1['START_POSITION'].isin(pos))
].copy()

# 2) 숫자형 변환
for c in ['PTS','player_poss']:
    det[c] = pd.to_numeric(det[c], errors='coerce').fillna(0)

# 3) 팀 전체 PPP = 총득점 / 총포제션
det_total_pts = det['PTS'].sum()
det_total_poss = det['player_poss'].sum()

det_ppp = det_total_pts / det_total_poss if det_total_poss > 0 else 0

print("=== 디트로이트 팀 PPP ===")
print(f"총득점: {det_total_pts}, 총포제션: {det_total_poss}")
print(f"팀 PPP: {det_ppp:.3f}")

# 경기+팀별 포제션 합
game_team_poss = df1.groupby(['GAME_ID','TEAM'], as_index=False)['player_poss'].sum()

# 팀별 경기 포제션 평균
avg_poss = game_team_poss['player_poss'].mean()
print("팀당 경기당 평균 포제션:", f"{avg_poss:.3f}")

data={
    '디트로이트 ppp':0.978,
    '상위팀 평균 ppp':1.028
}
ppp=pd.Series(data).reset_index()
ppp.columns = ['팀', 'PPP']
fig2=px.bar(
    ppp,
    x='팀',
    y='PPP' ,
    title="상위팀과 디트로이트 팀의 PTS per Possession 비교")

# 1) 상위팀 + 선발(C/F/G)만 필터 (TEAM_LVL 있을 때만)
if 'TEAM_LVL' in df1_named.columns:
    pos = ['C','F','G']
    top_starters = df1_named[(df1_named['TEAM_LVL'] == '상위팀') &
                       (df1_named['START_POSITION'].isin(pos))].copy()

    # 2) 상위팀 선발 전체의 TS% 평균
    top_starters_ts = top_starters['TS%'].mean()
    print("상위팀 선발 평균 TS%:", round(top_starters_ts, 3))

# 1) 선발 포지션 정의
pos = ['C','F','G']

# 2) 디트로이트 팀 + 선발만 필터
det_starters = df1_named[
    (df1_named['TEAM']=="Detroit") &
    (df1_named['START_POSITION'].isin(pos))
].copy()

# 3) TS% 평균 계산
det_starters_ts = det_starters['TS%'].mean()
print("디트로이트 선발 평균 TS%:", round(det_starters_ts, 3))

data={
    '디트로이트 선발 평균 TS%': 0.561,
    '상위팀 선발 평균 TS%': 0.591
}
ts=pd.Series(data).reset_index()
ts.columns = ['팀', 'TS%']
fig3=px.bar(
    ts,
    x='팀',
    y='TS%',
    title="상위팀 vs 디트로이트 팀 TS% 비교"
)

pos = ['C','F','G']
det_starters = df1_named[
    (df1_named['TEAM']=="Detroit") &
    (df1_named['START_POSITION'].isin(pos))
].copy()

# 선수별 TS% 평균 (포지션 포함)
det_player_ts = det_starters.groupby(['PLAYER_NAME','START_POSITION'], as_index=False)['TS%'].mean()

# 산점도 (TS% 크기로 반영)
fig4 = px.scatter(
    det_player_ts,
    x='PLAYER_NAME',
    y='TS%',
    color='START_POSITION',   # 포지션별 색 구분
    size='TS%',
    title="디트로이트 선발 선수별 TS% 분포 (포지션별 구분)",
    labels={'START_POSITION': 'Position'},   # 범례 제목 바꾸기
    category_orders={'START_POSITION': ['C','F','G']}  # 범례 순서 고정
)

fig4.for_each_trace(lambda t: t.update(
    name = t.name.replace("C","Center").replace("F","Forward").replace("G","Guard")
))

# 상위팀 TS% 산점도 (TEAM_LVL 있을 때만)
if 'TEAM_LVL' in df1_named.columns:
    pos = ['C','F','G']
    top_starters = df1_named[
        (df1_named['TEAM_LVL'] == '상위팀') &
        (df1_named['START_POSITION'].isin(pos))
    ].copy()

    top_player_ts = top_starters.groupby(['PLAYER_NAME','START_POSITION'], as_index=False)['TS%'].mean()

    fig5 = px.scatter(
        top_player_ts,
        x='PLAYER_NAME',
        y='TS%',
        color='START_POSITION',
        size='TS%',
        hover_data=['START_POSITION'],
        title="상위팀 선발 선수별 TS% 분포"
    )

    fig5.for_each_trace(lambda t: t.update(name = t.name.replace("C","Center").replace("F","Forward").replace("G","Guard")))
    fig5.update_traces(marker=dict(line=dict(width=1, color='black')))
    fig5.update_layout(xaxis_tickangle=45, yaxis_title="TS%", width=1000, height=600)

det = df1_named[df1_named['TEAM'].str.contains("Detroit", case=False, na=False)].copy()

# 상위팀만 필터 (TEAM_LVL 있을 때만)
if 'TEAM_LVL' in df1_named.columns:
    top = df1_named[df1_named['TEAM_LVL'] == '상위팀'].copy()

    # 평균 TO% 계산
    det_to_mean = det['TO%'].mean()
    top_to_mean = top['TO%'].mean()

    print("디트로이트 평균 TO%:", round(det_to_mean, 3))
    print("상위팀 평균 TO%:", round(top_to_mean, 3))

    data = {
        '팀': ['디트로이트', '상위팀'],
        'TO% 평균': [det_to_mean, top_to_mean]
    }
    df_plot = pd.DataFrame(data)

    fig6 = px.bar(df_plot,
                 x='팀',
                 y='TO% 평균',
                 title="디트로이트 vs 상위팀 TO% 평균 비교",
                 text='TO% 평균')

    fig6.update_traces(texttemplate='%{text:.3f}', textposition='outside')
    fig6.update_yaxes(range=[0,0.15])

# 선발 포지션
pos = ['C','F','G']

# 디트로이트 선발 선수만
det_starters = df1_named[
    (df1_named['TEAM']=="Detroit") &
    (df1_named['START_POSITION'].isin(pos))
]

# 선수별 평균 TO%
det_starters_to = det_starters.groupby('PLAYER_NAME', as_index=False)['TO%'].mean()

# 시각화
fig7 = px.bar(det_starters_to,
             x='PLAYER_NAME',
             y='TO%',
             title="디트로이트 선발 선수별 평균 TO%",
             text='TO%')
fig7.update_traces(texttemplate='%{text:.3f}', textposition='outside')
fig7.update_layout(xaxis_tickangle=45)
fig7.update_yaxes(range=[0,0.25])

# 디트로이트 선수만 추출
pos = ['C','F','G']

# 디트로이트 선발 선수만
det= df1_named[
    (df1_named['TEAM']=="Detroit") &
    (df1_named['START_POSITION'].isin(pos))
]
# 선수별 평균 AST%
det_ast = det.groupby(['PLAYER_NAME','START_POSITION'], as_index=False)['AST%'].mean()

# 산점도
fig = px.scatter(det_ast,
                 x='PLAYER_NAME',
                 y='AST%',
                 color='START_POSITION',  # 포지션별 색 구분
                 title="디트로이트 선수별 AST% 분포 (포지션별)",
                 size='AST%',             # 점 크기로 기여도 표현
                 hover_data=['START_POSITION'])

fig.update_layout(
    xaxis_tickangle=45,
    yaxis=dict(title="AST%"),
    width=950,
    height=600
)

# 상위팀만 필터 (TEAM_LVL 있을 때만)
if 'TEAM_LVL' in df1_named.columns:
    det = df1_named[df1_named['TEAM']=="Detroit"].copy()
    top = df1_named[df1_named['TEAM_LVL'] == '상위팀'].copy()

    # 평균 AST% 계산
    det_ast_mean = det['AST%'].mean()
    top_ast_mean = top['AST%'].mean()

    print("디트로이트 평균 AST%:", round(det_ast_mean, 3))
    print("상위팀 평균 AST%:", round(top_ast_mean, 3))

    # 시각화
    data = {
        '팀': ['디트로이트', '상위팀'],
        'AST% 평균': [det_ast_mean, top_ast_mean]
    }
    df_plot = pd.DataFrame(data)

    fig8 = px.bar(df_plot,
                 x='팀',
                 y='AST% 평균',
                 title="디트로이트 vs 상위팀 AST% 평균 비교",
                 text='AST% 평균')

    fig8.update_traces(texttemplate='%{text:.3f}', textposition='outside')
    fig8.update_yaxes(range=[0,7])

# 디트로이트 선수만 추출
pos = ['C','F','G']

# 디트로이트 선발 선수만
det= df1_named[
    (df1_named['TEAM']=="Detroit") &
    (df1_named['START_POSITION'].isin(pos))
]
# 선수별 평균 AST%
det_ast = det.groupby(['PLAYER_NAME','START_POSITION'], as_index=False)['AST%'].mean()

# 산점도
fig9 = px.scatter(det_ast,
                 x='PLAYER_NAME',
                 y='AST%',
                 color='START_POSITION',  # 포지션별 색 구분
                 title="디트로이트 선수별 AST% 분포 (포지션별)",
                 size='AST%',             # 점 크기로 기여도 표현
                 hover_data=['START_POSITION'])

fig9.update_layout(
    xaxis_tickangle=45,
    yaxis=dict(title="AST%"),
    width=950,
    height=600
)

# 상위팀 + 선발만 (TEAM_LVL 있을 때만)
if 'TEAM_LVL' in df1_named.columns:
    pos = ['C','F','G']
    top_starters = df1_named[
        (df1_named['TEAM_LVL'] == '상위팀') &
        (df1_named['START_POSITION'].isin(pos))
    ].copy()

    top_player_ast = top_starters.groupby(['PLAYER_NAME','START_POSITION'], as_index=False)['AST%'].mean()

    fig10 = px.scatter(
        top_player_ast,
        x='PLAYER_NAME',
        y='AST%',
        color='START_POSITION',   # 포지션별 색 구분
        size='AST%',              # AST% 크기 반영
        hover_data=['START_POSITION'],
        title="상위팀 선발 선수별 AST% 분포"
    )

    fig10.update_traces(marker=dict(line=dict(width=1, color='black')))
    fig10.update_layout(xaxis_tickangle=45, yaxis_title="AST%", width=1000, height=600)


''' 여기까지가 시각화 작업'''


"=================== html로 내보내기 ======================="
# 첫 그래프 (뼈대)
html: str = to_html(fig1, include_plotlyjs='cdn', full_html=True)

parts = [
    to_html(fig2, include_plotlyjs=False, full_html=False),
    to_html(fig3, include_plotlyjs=False, full_html=False),
]

# 중간에 삽입
insertion = "\n<hr>\n".join(parts)
lower = html.lower()
insert_at = lower.rfind("</body>")
html = html[:insert_at] + insertion + html[insert_at:]

out_path ='index.html'
Path(out_path).write_text(html, encoding='utf-8')

print(f"Saved: {out_path}")


/var/folders/vf/9p8syxcx1g77s8jzp8b8b8fh0000gn/T/ipykernel_78954/3138075267.py:60: DtypeWarning:

Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17 entries, 0 to 16
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PLAYER_NAME   17 non-null     object 
 1   PTS_per_poss  16 non-null     float64
dtypes: float64(1), object(1)
memory usage: 400.0+ bytes
None
=== 상위팀별 PPP ===
           TEAM       PPP
1      Brooklyn  1.099763
3        Denver  1.062903
0        Boston  1.062050
2     Cleveland  1.033071
7  Philadelphia  1.026947
6   New Orleans  1.001894
5     Milwaukee  0.997720
4       Memphis  0.976648

=== 상위팀 평균 PPP ===
1.033
=== 디트로이트 팀 PPP ===
총득점: 1259524, 총포제션: 1215905.52
팀 PPP: 1.036
팀당 경기당 평균 포제션: 112.503
상위팀 선발 평균 TS%: 0.6
디트로이트 선발 평균 TS%: 0.579
디트로이트 평균 TO%: 0.121
상위팀 평균 TO%: 0.14
디트로이트 평균 AST%: 16.163
상위팀 평균 AST%: 15.153
Saved: index.html
